In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os
import time
import plotly
from datetime import datetime
import optuna
import sklearn
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
SEED = 10
limit_rows = None

In [3]:
print(f"Execution started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

Execution started at 2022-03-02 16:51:45.


In [4]:
def load_csv(name):
    if limit_rows is None:
        full_filename = f'../Data Preprocessing/sklearn/full/iot23_combined_{name}.csv'
    else:
        full_filename = f'../Data Preprocessing/sklearn/semi/iot23_combined_{int(limit_rows/1000)}k_{name}.csv'
    
    df = pd.read_table(filepath_or_buffer=full_filename, header=None, sep=',').infer_objects().to_numpy()
    
    return df.ravel() if df.shape[1] == 1 else df

In [5]:
X_train, X_test, y_train, y_test = load_csv('X_train'), load_csv('X_test'), load_csv('y_train'), load_csv('y_test')

print('X_train',X_train.shape,'\ny_train',y_train.shape)
print('X_test',X_test.shape,'\ny_test',y_test.shape)

X_train (7142855, 15) 
y_train (7142855,)
X_test (1785714, 15) 
y_test (1785714,)


In [6]:
from sklearn.kernel_approximation import Nystroem
from sklearn.svm import LinearSVC

def objective(trial):
    
    # feature mapper parameters
    nystroem_gamma        = trial.suggest_discrete_uniform("nystroem_gamma", 0.1, 1.0, 0.1)
    nystroem_n_components = trial.suggest_int("nystroem_n_components", 50, 200, 25)

    feature_mapper = Nystroem(gamma=nystroem_gamma,n_components=nystroem_n_components)
    feature_mapper.fit(X_train)
    
    X_train_transf = feature_mapper.transform(X_train)
    X_test_transf  = feature_mapper.transform(X_test)
    
    # classifier parameters
    dual = trial.suggest_categorical('dual', [False])
    C    = trial.suggest_loguniform('C', 1e-6, 1e3)
    
    classifier_obj = LinearSVC(dual=dual,C=C)
    
    classifier_obj.fit(X_train_transf, y_train)
    
    y_pred = classifier_obj.predict(X_test_transf)
    
    accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
    
    return accuracy

In [7]:
print(f"Optimization started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

Optimization started at 2022-03-02 16:51:54.


In [ ]:
study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=10, n_jobs=2, catch=(NotImplementedError,))

[I 2022-03-02 16:51:54,286] A new study created in memory with name: no-name-c32c5f86-a65b-4625-a3a3-03925ad8390f
C:\Users\marce\anaconda3\envs\tf\lib\site-packages\optuna\study\study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(


In [ ]:
print(f"Optimization finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

In [ ]:
def format_trial(trial):
    return json.dumps({'number' : trial.number,
                       'score'  : trial.values[0],
                       'params' : trial.params}, indent=4, default=str)

In [ ]:
print(format_trial(study.best_trial))

In [ ]:
for trial in study.best_trials:
    print(format_trial(trial))

In [ ]:
optuna.visualization.plot_optimization_history(study).show()